_Last Updated: 06/21/2021_

In [1]:
import gurobipy as gp
from gurobipy import GRB, abs_
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import pandas as pd
import json
%matplotlib inline

### Langrangian Subproblems

In [2]:
w_ub, w_lb, b_ub, b_lb = [1,-1,1,-1]
epsilon = 0.01

In [1]:
def zeta_0(λ):

    # Create a new model
    m = gp.Model("0-th layer")
    
    # Create variables
    
    alpha = {}
    beta = {}
    h = {}

    for k in range(K):
        beta[(k,0)] = m.addVar(lb=b_lb, ub=b_ub, vtype=GRB.CONTINUOUS, name="beta"+str((k,0)))
        for d in range(D):
            alpha[(d,k,0)] = m.addVar(lb=w_lb, ub=w_ub, vtype=GRB.CONTINUOUS, name="alpha"+str((d,k,0)))
            
        for n in range(N):
            h[(n,k,0)] = m.addVar(vtype=GRB.BINARY, name="h"+str((n,k,0)))
            
    # Set objective

    m.setObjective( 
        sum( sum( sum( -λ[(n,k_prime,k,1)]*h[(n,k_prime,0)] 
                      for k in range(K)) for k_prime in range(K)) for n in range(N)), GRB.MINIMIZE)
        
   # Add constraints 
    
    for n in range(N):
        for k in range(K):
            m.addConstr(sum(alpha[(d,k,0)]*x[n,d] for d in range(D)) + beta[(k,0)] 
                        <= (D*w_ub+b_ub + epsilon)*h[(n,k,0)], name="C1 Binary Neuron "+str((n,k,0)))
            
            m.addConstr(sum(alpha[(d,k,0)]*x[n,d] for d in range(D)) + beta[(k,0)] 
                        >= epsilon + (D*w_lb+b_lb - epsilon)*(1-h[(n,k,0)]), name="C2 Binary Neuron "+str((n,k,0)))
            
    # Optimize
    
    m.setParam('OutputFlag', 0) # uncomment to silence the output
    m.optimize()
    m.printQuality()
            
    for v in m.getVars():
#         print('%s %s %g' % (v.varName, "=", np.round(v.x, 3)))
        output[v.varName] = v.x 

#     print('Obj: %g' % m.objVal)
    
    return(m.objVal)


In [2]:
def zeta_l(layer, λ):
    
    # Create a new model
    m = gp.Model("l-th layer")
    
    l = layer # layer to be solved for
    
    # Create variables

    alpha = {}
    beta = {}
    z = {}
    h = {}
    g = {}
    
    for k in range(K):
        for n in range(N):
            for k_prime in range(K):
                z[(n,k_prime,k,l)] = m.addVar(lb=w_lb, ub=w_ub, vtype=GRB.CONTINUOUS, name= "z"+str((n,k_prime,k,l)))
                g[(n,k_prime,k,l)] = m.addVar(vtype=GRB.BINARY, name="g"+str((n,k_prime,k,l)))
                
            h[(n,k,l)] = m.addVar(vtype=GRB.BINARY, name="h"+str((n,k,l)))
        
        for k_prime in range(K):
            alpha[(k_prime,k,l)] = m.addVar(lb=w_lb, ub=w_ub, vtype=GRB.CONTINUOUS, name="alpha"+str((k_prime,k,l)))
        
        beta[(k,l)] = m.addVar(lb=b_lb, ub=b_ub, vtype=GRB.CONTINUOUS, name="beta"+str((k,l)))
        
    # Set objective
        
    m.setObjective( sum( sum( sum( λ[(n,k_prime,k,l)]*g[(n,k_prime,k,l)] 
                                 - λ[(n,k_prime,k,l+1)]*h[(n,k_prime,l)] 
                                 for k in range(K)) for k_prime in range(K)) for n in range(N)) ,GRB.MINIMIZE)
    
    # Add constraints
    
    for n in range(N):
        for k in range(K):
            m.addConstr(sum(z[(n,k_prime,k,l)] for k_prime in range(K)) + beta[(k,l)] 
                        <= (K*w_ub+b_ub + epsilon)*h[(n,k,l)], name="C1 Binary Neuron "+str((n,k,l))) 

            m.addConstr(sum(z[(n,k_prime,k,l)] for k_prime in range(K)) + beta[(k,l)] 
                        >= epsilon + (K*w_lb+b_lb - epsilon)*(1-h[(n,k,l)]), name="C2 Binary Neuron "+str((n,k,l)))
            
            for k_prime in range(K):
                m.addConstr(z[(n,k_prime,k,l)] <= alpha[(k_prime,k,l)] + (-w_lb)*(1-g[(n,k_prime,k,l)]), 
                            name="z-alpha Upper Bound "+str((n,k_prime,k,l)))
                m.addConstr(z[(n,k_prime,k,l)] >= alpha[(k_prime,k,l)] + (-w_ub)*(1-g[(n,k_prime,k,l)]), 
                            name="z-alpha Lower Bound"+str((n,k_prime,k,l)))
                m.addConstr(z[(n,k_prime,k,l)] <= (w_ub)*g[(n,k_prime,k,l)], name = "z-g Upper Bound"+str((n,k_prime,k,l)))
                m.addConstr(z[(n,k_prime,k,l)] >= (w_lb)*g[(n,k_prime,k,l)], name = "z-g Lower Bound"+str((n,k_prime,k,l)))
                
    # Optimize
    
    m.setParam('OutputFlag', 0) # uncomment to silence the output
    m.optimize()
    m.printQuality()
            
    for v in m.getVars():
#         print('%s %s %g' % (v.varName, "=", np.round(v.x, 3)))
        output[v.varName] = v.x 

#     print('Obj: %g' % m.objVal)
    
    return(m.objVal)


In [3]:
def zeta_penultimate_L(λ):
    
    # Create a new model
    m = gp.Model("(L-1)st layer")
    
    # Create variables

    alpha = {}
    beta = {}
    z = {}
    h = {}
    g = {}
    
    for k in range(K):
        for n in range(N):
            for k_prime in range(K):
                z[(n,k_prime,k,L-1)] = m.addVar(lb=w_lb, ub=w_ub, vtype=GRB.CONTINUOUS, name= "z"+str((n,k_prime,k,L-1)))
                g[(n,k_prime,k,L-1)] = m.addVar(vtype=GRB.BINARY, name="g"+str((n,k_prime,k,L-1)))
                
            h[(n,k,L-1)] = m.addVar(vtype=GRB.BINARY, name="h"+str((n,k,L-1)))
        
        for k_prime in range(K):
            alpha[(k_prime,k,L-1)] = m.addVar(lb=w_lb, ub=w_ub, vtype=GRB.CONTINUOUS, name="alpha"+str((k_prime,k,L-1)))
        
        beta[(k,L-1)] = m.addVar(lb=b_lb, ub=b_ub, vtype=GRB.CONTINUOUS, name="beta"+str((k,L-1)))
        
    # Set objective
        
    m.setObjective( sum( sum( sum( λ[(n,k_prime,k,L-1)]*g[(n,k_prime,k,L-1)] for k in range(K))
                             - sum( λ[(n,k_prime,j,L)]*h[(n,k_prime,L-1)] for j in range(J))
                             for k_prime in range(K)) for n in range(N))
                   ,GRB.MINIMIZE)
    
    # Add constraints
    
    for n in range(N):
        for k in range(K):
            m.addConstr(sum(z[(n,k_prime,k,L-1)] for k_prime in range(K)) + beta[(k,L-1)] 
                        <= (K*w_ub+b_ub + epsilon)*h[(n,k,L-1)], name="C1 Binary Neuron "+str((n,k,L-1))) 

            m.addConstr(sum(z[(n,k_prime,k,L-1)] for k_prime in range(K)) + beta[(k,L-1)] 
                        >= epsilon + (K*w_lb+b_lb - epsilon)*(1-h[(n,k,L-1)]), name="C2 Binary Neuron "+str((n,k,L-1)))
            
            for k_prime in range(K):
                m.addConstr(z[(n,k_prime,k,L-1)] <= alpha[(k_prime,k,L-1)] + (-w_lb)*(1-g[(n,k_prime,k,L-1)]),
                            name="z-alpha Upper Bound "+str((n,k_prime,k,L-1)))
                m.addConstr(z[(n,k_prime,k,L-1)] >= alpha[(k_prime,k,L-1)] + (-w_ub)*(1-g[(n,k_prime,k,L-1)]), 
                            name="z-alpha Lower Bound"+str((n,k_prime,k,L-1)))
                m.addConstr(z[(n,k_prime,k,L-1)] <= (w_ub)*g[(n,k_prime,k,L-1)], name = "z-g Upper Bound"+str((n,k_prime,k,L-1)))
                m.addConstr(z[(n,k_prime,k,L-1)] >= (w_lb)*g[(n,k_prime,k,L-1)], name = "z-g Lower Bound"+str((n,k_prime,k,L-1)))
                
    # Optimize
    
    m.setParam('OutputFlag', 0) # uncomment to silence the output
    m.optimize()
    m.printQuality()
            
    for v in m.getVars():
#         print('%s %s %g' % (v.varName, "=", np.round(v.x, 3)))
        output[v.varName] = v.x 

#     print('Obj: %g' % m.objVal)
    
    return(m.objVal)


In [4]:
def zeta_L(λ):
    
    # Create a new model
    m = gp.Model("L-th layer")
        
    # Create variables
    
    alpha = {}
    beta = {}
    z = {}
    g = {}
    y_hat = {}
    loss_prime = {}
    loss = {}

    for n in range(N):
        
        loss[n] = m.addVar(vtype=GRB.BINARY, name="loss"+str(n))
        
        for j in range(J):
            y_hat[(n,j)] = m.addVar(vtype=GRB.BINARY, name="y_hat"+str((n,j)))
            loss_prime[(n,j)] = m.addVar(lb=0, ub=1, vtype=GRB.CONTINUOUS, name="loss_prime"+str((n,j)))
        
            for k_prime in range(K):
                z[(n,k_prime,j,L)] = m.addVar(lb=w_lb, ub=w_ub, vtype=GRB.CONTINUOUS, name="z"+str((n,k_prime,j,L)))
                g[(n,k_prime,j,L)] = m.addVar(vtype=GRB.BINARY, name="g"+str((n,k_prime,j,L)))
    
    for j in range(J):
    
        beta[(j,L)] = m.addVar(lb=b_lb, ub=b_ub, vtype=GRB.CONTINUOUS, name="beta"+str((j,L)))

        for k_prime in range(K):
            alpha[(k_prime,j,L)] = m.addVar(lb=w_lb, ub=w_ub, vtype=GRB.CONTINUOUS, name="alpha"+str((k_prime,j,L)))
            
    # Set objective
    
    m.setObjective( sum( loss[n] 
                        + sum( sum( λ[(n,k_prime,j,L)]*g[(n,k_prime,j,L)] 
                                   for j in range(J)) for k_prime in range(K))
                       for n in range(N)) ,GRB.MINIMIZE)
    
    # Add constraints
    
    for n in range(N):
        for j in range(J):
            m.addConstr(sum(z[(n,k_prime,j,L)] for k_prime in range(K)) + beta[(j,L)] 
                        <= ((K*w_ub)+b_ub + epsilon)*y_hat[(n,j)], name="C1 Binary Neuron "+str((n,j,L))) 

            m.addConstr(sum(z[(n,k_prime,j,L)] for k_prime in range(K)) + beta[(j,L)] 
                        >= epsilon + ((K*w_lb)+b_lb - epsilon)*(1-y_hat[(n,j)]), name="C2 Binary Neuron "+str((n,j,L)))

            m.addConstr(loss_prime[(n,j)] >= y[n][j] - y_hat[(n,j)], name = "C1 Abs Diff"+str((n,j)))
            m.addConstr(loss_prime[(n,j)] >= -y[n][j] + y_hat[(n,j)], name = "C2 Abs Diff"+str((n,j)))
            
        m.addConstr(sum(loss_prime[(n,j)] for j in range(J)) <= 1 - epsilon + ((J-1) + epsilon)*loss[n], name = "Loss Function"+str(n))
        
        for k_prime in range(K):
            for j in range(J):
                m.addConstr(z[(n,k_prime,j,L)] <= alpha[(k_prime,j,L)] + (-w_lb)*(1-g[(n,k_prime,j,L)]),
                            name="z-alpha Upper Bound "+str((n,k_prime,j,L)))
                m.addConstr(z[(n,k_prime,j,L)] >= alpha[(k_prime,j,L)] + (-w_ub)*(1-g[(n,k_prime,j,L)]), 
                                name="z-alpha Lower Bound"+str((n,k_prime,j,L)))
                m.addConstr(z[(n,k_prime,j,L)] <= (w_ub)*g[(n,k_prime,j,L)], name = "z-g Upper Bound"+str((n,k_prime,j,L)))
                m.addConstr(z[(n,k_prime,j,L)] >= (w_lb)*g[(n,k_prime,j,L)], name = "z-g Lower Bound"+str((n,k_prime,j,L)))
                            
    # Optimize
    
    m.setParam('OutputFlag', 0) # uncomment to silence the output
    m.optimize()
    m.printQuality()
            
    for v in m.getVars():
#         print('%s %s %g' % (v.varName, "=", np.round(v.x, 3)))
        output[v.varName] = v.x 

#     print('Obj: %g' % m.objVal)
    
    return(m.objVal)


### Solving the dual with the Subgradient algorithm

##### Need to modify terminating condition

In [ ]:

    # First iteration

        ## initialize langrange multiplier

λ = {} 

np.random.seed(6)

for n in range(N):
    for k_prime in range(K):
        for k in range(K):
            for l in range(1,L):
                λ[(n,k_prime,k,l)] = np.random.uniform(low=-100, high=100)
        for j in range(J):
            λ[(n,k_prime,j,L)] = np.random.uniform(low=-100, high=100)

λ_array = list(λ.values())        

output = {} # dictionary of outputs

        ## Solve subproblems and find the dual objective

dual_obj = zeta_0(λ) + sum(zeta_l(l,λ) for l in range(1,L-1)) + zeta_penultimate_L(λ) + zeta_L(λ) 

        ## Find the subgradient vector at λ

s = {}

for n in range(N):
    for k_prime in range(K):
        for k in range(K):
            for l in range(1,L):
                s[(n,k_prime,k,l)] = output['g'+str((n,k_prime,k,l))] - output['h'+str((n,k_prime,l-1))]
        for j in range(J):
            s[(n,k_prime,j,L)] = output['g'+str((n,k_prime,j,L))] - output['h'+str((n,k_prime,L-1))]

s_array = np.array(list(s.values()))

        ## Adaptive Polyak Stepsize

mu_naught = 2
mu = mu_naught

Z_best = 0 # Lower bound on primal

gamma = mu*((Z_best - dual_obj)/(np.linalg.norm(s_array))**2)

        ## Track the projected subgradinet

indicator = []

for i in range(len(λ_array)):
    if (λ_array[i] + gamma*s_array[i]) > 0:
        indicator.append(1)
    else:
        indicator.append(0)

indicator = np.array(indicator)
projected_s = indicator.T*s_array

subgradient_tracker = [np.linalg.norm(s_array)]
dual_obj_tracker = [dual_obj]
gamma_tracker = [gamma]
projected_s_tracker = [np.linalg.norm(projected_s)]
mu_tracker = [mu]

    # Iterative loops until we meet terminating condition

adapt_param_factor = 2/3 # try 1/4 and 2/3

T = 0 # Number of times Z_{D} did not increase

for t in range(1,1000): 

    if t%100 == 0:
        print("Iteration count:", t)

        ## Update λ

    for n in range(N):
        for k_prime in range(K):
            for k in range(K):
                for l in range(1,L):
                    λ[(n,k_prime,k,l)] = λ[(n,k_prime,k,l)] + gamma*s[(n,k_prime,k,l)]
            for j in range(J):
                λ[(n,k_prime,j,L)] = λ[(n,k_prime,j,L)] + gamma*s[(n,k_prime,j,L)]

    λ_array = np.array(list(λ.values()))

        ## Solve subproblems and find the dual objective

    dual_obj = zeta_0(λ) + sum(zeta_l(l,λ) for l in range(1,L-1)) + zeta_penultimate_L(λ) + zeta_L(λ) 

    dual_obj_tracker.append(dual_obj)

        ## Find the subgradient vector at λ

    for n in range(N):
        for k_prime in range(K):
            for k in range(K):
                for l in range(1,L):
                    s[(n,k_prime,k,l)] = output['g'+str((n,k_prime,k,l))] - output['h'+str((n,k_prime,l-1))]
            for j in range(J):
                s[(n,k_prime,j,L)] = output['g'+str((n,k_prime,j,L))] - output['h'+str((n,k_prime,L-1))]

    s_array = np.array(list(s.values()))
    subgradient_tracker.append(np.linalg.norm(s_array))

        ## Adaptive Polyak Stepsize

    if dual_obj > Z_best:
        Z_best = dual_obj

    if not (dual_obj > dual_obj_tracker[-2]):
        T += 1
    else:
        T = 0

    if T >= 2: # scaling mu if Z_{D} did not increase in the last 2 iterations
        mu = adapt_param_factor*mu

    mu_tracker.append(mu)

    gamma = mu*((Z_best - dual_obj)/(np.linalg.norm(s_array))**2)
    gamma_tracker.append(gamma)

        ## Track norm of projected subgradinet

    indicator = []

    for i in range(len(λ_array)):
        if (λ_array[i] + gamma*s_array[i]) < 0:
            indicator.append(0)
        else:
            indicator.append(1)

    indicator = np.array(indicator)
    projected_s = indicator.T*s_array

    projected_s_tracker.append(np.linalg.norm(projected_s))

    # Choose the right terminating condition
    
    if (dual_obj >= -1e-5) and (dual_obj_tracker[t]-dual_obj_tracker[t-1] < 1e-6): 
        break

#         The lower bound on the primal might not always be 0, but should be??


###  Plot Dual Objective and Subgradient figures

In [ ]:
fig, (ax1, ax2) = plt.subplots(2)
fig.set_size_inches(12, 12)

subgrad_y1 = [np.mean(subgradient_tracker[i:i+10]) for i in range(0,len(subgradient_tracker),10)]
# subgrad_y2 = [np.mean(projected_s_tracker[i:i+10]) for i in range(0,len(projected_s_tracker),10)]

ax1.plot([x for x in range(t+1)], dual_obj_tracker, c="dodgerblue")
ax2.plot([x for x in range(len(subgrad_y1))], subgrad_y1, c="indianred")
# ax2.plot([x for x in range(len(projected_s_tracker))], projected_s_tracker, c="dodgerblue")

ax1.set_title("Dual Objective (μ0 = {}, α = {})".format(np.round(mu_naught,2), np.round(adapt_param_factor,2)), fontsize=14)
ax2.set_title("10-Norm-of-Subgradient Averages (μ0 = {}, α = {})".format(np.round(mu_naught,2), np.round(adapt_param_factor,2)), fontsize=14)

ax2.legend(["Subgradient", "Projected Subgradient"])

ax1.set_xlabel('Iteration Count', fontsize=14)
ax2.set_xlabel('Time Averaged-Counts', fontsize=14)

# filename = 'mu0={} alpha={}.png'.format(np.round(mu_naught,2), np.round(adapt_param_factor,2))
# plt.savefig("Subgradient Plots/"+filename)

None

In [6]:
# TODO:
## Get MNIST data and run subgradient algorithm
## Use pre-trained parameters to get a primal solution using SGD and MIP
### Construct MNIST appropriate networks for SGD and MIP